In [1]:
import shutil
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import imageio.v3
import time
import cv2
import csv
from numpy import linalg 
from PIL import Image 
import matplotlib.pyplot as plt
from skimage.color import label2rgb
import imageio.v3 as iio
from skimage import filters
#from skimage.color import rgb2gray          
from skimage.measure import regionprops

In [8]:
input_folder_phasemask = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\First_200_check\phasemask'
output_folder_phasemask = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\phasemask_preprocessed'

camera1_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\camera_1_predict'
camera2_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\camera_1_predict'

#crop
reference_image_focus = iio.imread(r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\reference_image_focus_vortex.png')
reference_image_defocus = iio.imread(r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\reference_image_defocus_vortex.png')

origin_camera1_crop = camera1_path
target_camera1_crop = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\camera_2_crop'

origin_camera2_crop = camera2_path
target_camera2_crop = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datenaufnahme\Raw_Data_Ditzingen_Vortex_Abb08_New\camera_2_crop'

Preprocess Phasemask

In [3]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]


if not os.path.exists(output_folder_phasemask):
    os.makedirs(output_folder_phasemask)

try:
    files = [file for file in os.listdir(input_folder_phasemask) if file.lower().endswith('.png')]
    sorted_files = sorted(files, key=natural_sort_key)

    for idx, filename in enumerate(sorted_files, start=1):
        new_filename = f"{idx}_idx_{filename}"
        input_img_path = os.path.join(input_folder_phasemask, filename)
        output_img_path = os.path.join(output_folder_phasemask, new_filename)
        
        shutil.copy2(input_img_path, output_img_path)
        

    print("Completed transferring images with indexed filenames.")
except Exception as e:
    print(f"An error occurred: {e}")

Completed transferring images with indexed filenames.


In [4]:
# Cropping the phasemask in the output_folder_phasemask
from methods import crop_around_center


for filename in os.listdir(output_folder_phasemask):
    if filename.lower().endswith(('.png')):
        
        image_path = os.path.join(output_folder_phasemask, filename)
        save_path = os.path.join(output_folder_phasemask, filename)
        
        try:
            crop_around_center(image_path, save_path)
        except Exception as e:
            print(f"Error processing image {filename}: {e}")

Delete Oversaturated images

In [5]:
from PIL import Image
import numpy as np

def oversaturation(image_path, pixel_count_threshold=10): #Analyse a few oversaturated images.
    img = Image.open(image_path)
    img_array = np.array(img)
    oversaturated_pixels = np.sum(img_array == 255)
    return oversaturated_pixels > pixel_count_threshold

def delete_oversaturated_images(folder_path, pixel_count_threshold=10):
    files = os.listdir(folder_path)
    for filename in files:
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            image_path = os.path.join(folder_path, filename)
            if oversaturation(image_path, pixel_count_threshold):
                os.remove(image_path)
                #print(f"Deleted oversaturated image: {filename}")

In [9]:
delete_oversaturated_images(camera1_path)
delete_oversaturated_images(camera2_path)

In [10]:
import os
from concurrent.futures import ThreadPoolExecutor

ordner1 = camera1_path
ordner2 = camera2_path
ordner3 = output_folder_phasemask

# Listen der Dateien in jedem Ordner erstellen
dateien_ordner1 = set(os.listdir(ordner1))
dateien_ordner2 = set(os.listdir(ordner2))
dateien_ordner3 = set(os.listdir(ordner3))

def loesche_dateien(index):
    dateiname = f"{index}_idx_"

    pfad1 = next((os.path.join(ordner1, f) for f in dateien_ordner1 if f.startswith(dateiname)), None)
    pfad2 = next((os.path.join(ordner2, f) for f in dateien_ordner2 if f.startswith(dateiname)), None)
    pfad3 = next((os.path.join(ordner3, f) for f in dateien_ordner3 if f.startswith(dateiname)), None)
    
    if not (pfad1 and pfad2 and pfad3):
        if pfad1:
            os.remove(pfad1)
            #print(f"Gelöscht: {pfad1}")
        if pfad2:
            os.remove(pfad2)
            #print(f"Gelöscht: {pfad2}")
        if pfad3:
            os.remove(pfad3)
            #print(f"Gelöscht: {pfad3}")

def main():
    n = 25000  # Anzahl der Indizes

    with ThreadPoolExecutor() as executor:
        executor.map(loesche_dateien, range(1, n + 1))

if __name__ == "__main__":
    main()


Noise Filtering

In [12]:
import os
import imageio
from PIL import Image
import numpy as np
from methods import background_substraction

In [14]:
for filename in os.listdir(camera1_path):
    if filename.endswith('.png'):
        input_img_path = os.path.join(camera1_path, filename)
        output_img_path = os.path.join(camera1_path, filename)
        #print(f'Processing {input_img_path}')

        
        processed_image = background_substraction(input_img_path)
        processed_image = processed_image.convert('L')
        processed_image.save(output_img_path)

In [15]:
for filename in os.listdir(camera2_path):
    if filename.endswith('.png'):
        input_img_path = os.path.join(camera2_path, filename)
        output_img_path = os.path.join(camera2_path, filename)
        #print(f'Processing {input_img_path}')

        
        processed_image = background_substraction(input_img_path)
        processed_image = processed_image.convert('L')
        processed_image.save(output_img_path)

# Crop images

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

focus image

In [ ]:
img_array = np.asarray(reference_image_focus)

#image = img_array / 255.0
print(f"Image_sub_background: {img_array}")
plt.title("Image_sub_background")
plt.imshow(img_array)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, color
from skimage.color import label2rgb
from skimage.measure import regionprops
# img_sub_background
print(f"Shape: {reference_image_focus.shape}")
print(f"Max_image_sub_background: {np.max(reference_image_focus)}")
image = reference_image_focus

# If the input image has thgree channels
image = color.rgb2gray(reference_image_focus) 

threshold_value = filters.threshold_otsu(image)
#threshold_value = 0.65 # for gaussian camera 1

# image with only 1 and 0
labeled_foreground = (image > threshold_value).astype(int)
print(np.max(labeled_foreground))
properties = regionprops(labeled_foreground, image)
center_of_mass = properties[0].centroid
weighted_center_of_mass = properties[0].weighted_centroid
print(f"(y,x) {center_of_mass}")


#Plot the image then 3 channels
colorized = label2rgb(labeled_foreground, image, colors=['black', 'red'], alpha=0.1)
fig, ax = plt.subplots()
ax.imshow(colorized)




# Note the inverted coordinates because plt uses (x, y) while NumPy uses (row, column)
ax.scatter(center_of_mass[1], center_of_mass[0], s=160, c='C0', marker='+')
plt.show()


In [ ]:
center_of_mass_final=[center_of_mass[0],center_of_mass[1]]

print(center_of_mass_final)

In [ ]:
from methods import crop_beam
image_cropped = crop_beam(image,center_of_mass_final, crop_size=512)
print(type(image_cropped))
plt.title("Beam_cropped")
plt.imshow(image_cropped)
plt.show()

In [ ]:
for filename in os.listdir(origin_camera1_crop):
    if filename.lower().endswith(('.bmp','.png')):
        full_path = os.path.join(origin_camera1_crop, filename)
        target_path = os.path.join(target_camera1_crop, filename)

        
        if os.path.exists(target_path):
            #print(f"{filename} already exists in the target folder, skipping.")
            continue
        
        
        image = cv2.imread(full_path)
        cropped_image = crop_beam(image,center_of_mass=center_of_mass_final,crop_size=512)
        cv2.imwrite(target_path, cropped_image) 
        print("save")
        
print("All images have been successfully cropped and saved in the target folder.")

defocus camera

In [ ]:
img_array = np.asarray(reference_image_defocus)

#image = img_array / 255.0
print(f"Image_sub_background: {img_array}")
plt.title("Image_sub_background")
plt.imshow(img_array)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, color
from skimage.color import label2rgb
from skimage.measure import regionprops
# img_sub_background
print(f"Shape: {reference_image_focus.shape}")
print(f"Max_image_sub_background: {np.max(reference_image_focus)}")
image = reference_image_focus

# If the input image has thgree channels
image = color.rgb2gray(reference_image_focus) 

threshold_value = filters.threshold_otsu(image)
#threshold_value = 0.65 # for gaussian camera 1

# image with only 1 and 0
labeled_foreground = (image > threshold_value).astype(int)
print(np.max(labeled_foreground))
properties = regionprops(labeled_foreground, image)
center_of_mass = properties[0].centroid
weighted_center_of_mass = properties[0].weighted_centroid
print(f"(y,x) {center_of_mass}")


#Plot the image then 3 channels
colorized = label2rgb(labeled_foreground, image, colors=['black', 'red'], alpha=0.1)
fig, ax = plt.subplots()
ax.imshow(colorized)




# Note the inverted coordinates because plt uses (x, y) while NumPy uses (row, column)
ax.scatter(center_of_mass[1], center_of_mass[0], s=160, c='C0', marker='+')
plt.show()


In [ ]:
center_of_mass_final=[center_of_mass[0],center_of_mass[1]]

print(center_of_mass_final)

In [ ]:
from methods import crop_beam
image_cropped = crop_beam(image,center_of_mass_final, crop_size=512)
print(type(image_cropped))
plt.title("Beam_cropped")
plt.imshow(image_cropped)
plt.show()

In [ ]:
for filename in os.listdir(origin_camera2_crop):
    if filename.lower().endswith(('.bmp','.png')):
        full_path = os.path.join(origin_camera2_crop, filename)
        target_path = os.path.join(target_camera2_crop, filename)

        
        if os.path.exists(target_path):
            #print(f"{filename} already exists in the target folder, skipping.")
            continue
        
        
        image = cv2.imread(full_path)
        cropped_image = crop_beam(image,center_of_mass=center_of_mass_final,crop_size=512)
        cv2.imwrite(target_path, cropped_image) 
        print("save")
        
print("All images have been successfully cropped and saved in the target folder.")